<a href="https://colab.research.google.com/github/shiridikumar/AAD_project/blob/master/NLP_Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import nltk
import pandas as pd
import numpy as np

In [5]:
tweets=[]
tweets_corpus=open("/content/drive/MyDrive/intro_to_nlp_asign1/general-tweets.txt","r")
for i in tweets_corpus.readlines():
  tweets.append(i);
tweets=np.array(tweets)

In [108]:
#for URL substitutuion
import re

text=tweets[20]
def tokenize(text):
  x = re.sub("https?:\/\/[a-zA-Z0-9]+\.[a-zA-Z0-9@:%._\+~#=><!]+(\/[a-zA-Z0-9@:%._\+~#=\?]+)*" ,"<URL>",text)
  text=x

  #for Hshtag substituion
  x=re.sub("\s#([^\s^\n])*|^#([^\s^\n])*"," <HASHTAG>",text)
  text=x


  #for mentions
  x=re.sub("\s@[^\s^\n]*|^@[^\s^\n]*"," <MENTION>",text)
  text=x

  #for removing redundant punctuations
  a=list("@:%_~#=><!,~-")
  for i in range(len(a)):
    string="{}+".format(a[i])
    x=re.sub(string,a[i],text)
    text=x

  special_char=list(".?^%$*+/()")
  for i in range(len(special_char)):
    string="\{}+".format(special_char[i])
    x=re.sub(string,"{}".format(special_char[i]),text)
    text=x
  

  text=re.sub("[\s]+"," ",text)
  tokens=text.split()
  for i in range(len(tokens)):
    if(tokens[i]!= "<MENTION>" and tokens[i]!="<URL>" and tokens[i]!="<HASHTAG>" and tokens[i]!="<s>" and tokens[i]!="<\s>"):

      for j in range(len(a)):
        string="([\w]+){}".format(a[j])
        tokens[i]=re.sub(string,r"\1 {}".format(a[j]),tokens[i])
        string="{}([\w]+)".format(a[j])
        tokens[i]=re.sub(string,r"{} \1".format(a[j]),tokens[i])
        


      for j in range(len(special_char)):
        string="([\w]+)\{}".format(special_char[j])
        tokens[i]=re.sub(string,r"\1 {}".format(special_char[j]),tokens[i])

        string="\{}([\w]+)".format(special_char[j])
        tokens[i]=re.sub(string,r"{} \1".format(special_char[j]),tokens[i])


  tokens=" ".join(tokens)
  tokens="<s> "+tokens+" </s>"
  tokens=tokens.lower()
  text=re.sub("[\s]+"," ",tokens)
  tokens=text.split()
  return tokens

In [7]:
text="#ieroween THE STORY OF IEROWEEN! THE VIDEO ->>>>>>>>>>>>>>>>>>>>>> http://bit.ly/2VFPAV <<<< JUST FOR FRANK !!! Ã§ %@&*@#$!@# #asdasda asda@asda$%^$3"
print(tokenize("i can do anything for anyone"))

['<s>', 'i', 'can', 'do', 'anything', 'for', 'anyone', '</s>']


In [8]:
text="hello!sdasdasda"
x=re.sub("!"," ! ",text)
print(x)

hello ! sdasdasda


In [9]:
europarl=[]
europarl_corpus=open("/content/drive/MyDrive/intro_to_nlp_asign1/europarl-corpus.txt","r")
for i in europarl_corpus.readlines():
  europarl.append(i)
europarl=np.array(europarl)

In [10]:
ngrams=[]
europarltokens=[""]*len(europarl)
n=2
for i in range(len(europarl)):
  europarltokens[i]=tokenize(europarl[i])

In [153]:
print(europarl[1])

I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.



In [58]:
def get_ngrams(n,text):
  ngrams=list(zip(*[text[i:] for i in range(n)]))
  return ngrams

In [59]:
get_ngrams(4,tokenize(tweets[13]))

[('<s>', 'well', 'i', "can't"),
 ('well', 'i', "can't", 'spell'),
 ('i', "can't", 'spell', 'it'),
 ("can't", 'spell', 'it', 'out'),
 ('spell', 'it', 'out', 'for'),
 ('it', 'out', 'for', 'you'),
 ('out', 'for', 'you', '.'),
 ('for', 'you', '.', "it's"),
 ('you', '.', "it's", 'never'),
 ('.', "it's", 'never', 'gonna'),
 ("it's", 'never', 'gonna', 'be'),
 ('never', 'gonna', 'be', 'that'),
 ('gonna', 'be', 'that', 'simple'),
 ('be', 'that', 'simple', '.'),
 ('that', 'simple', '.', '</s>')]

In [ ]:
for i in range(10):
  print(europarltokens[i])

In [76]:
vocab=set()
for i in range(len(europarltokens)):
  vocab=set(europarltokens[i]).union(vocab)

In [129]:
ngrams=[]
for i in range(len(europarltokens)):
  ngrams.extend(get_ngrams(2,europarltokens[i]))

nminus=[]
for i in range(len(europarltokens)):
  nminus.extend(get_ngrams(1,europarltokens[i]))

#Just viewing first 10 ngrams

In [130]:
for i in range(10):
  print(ngrams[i])

('<s>', 'resumption')
('resumption', 'of')
('of', 'the')
('the', 'session')
('session', '</s>')
('<s>', 'i')
('i', 'declare')
('declare', 'resumed')
('resumed', 'the')
('the', 'session')


#Counts of n grams

In [131]:
from collections import Counter
count_ngrams=Counter(ngrams)


#Counts of n-1 grams

In [132]:
count_nminus=Counter(nminus)


#Total number of ngrams which have count of 1


In [133]:
count_values=list(count_ngrams.values())
n1=count_values.count(1)

#Kneyser ney Smoothing

In [156]:
total_prob=1
total_ngrams=len(vocab)*len(vocab)
a="I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period."
a=get_ngrams(2,tokenize(a))
N=len(count_ngrams)
effective_n0_prob=(n1/N)
for i in range(len(a)):
  if(count_ngrams[a[i]]==0):
    total_prob*=effective_n0_prob
    print(a[i])
    print("********")
  else:
    denominator=a[i][:-1]
    discount_prob=max(count_ngrams[a[i]]-0.75,0)/count_nminus[denominator]
    #calculating p_continuation
    pconts=len(list(filter(lambda x:x[-1]==a[i][-1],count_ngrams)))
    pcont_prob=pconts/N
    #finding lambda value
    k=0.75/len(list(filter(lambda x:x[:-1]==a[i][:-1],count_ngrams)))

    effective_prob=discount_prob+k*pcont_prob
    print(effective_prob,discount_prob)
    total_prob*=effective_prob
print("{}".format(total_prob))
print((1/(total_prob))**(1/len(a)))

0.11066371850898644 0.1106625
0.0009150614507106 0.0009149529452771
0.23214633512952024 0.23214285714285715
0.19365610117967852 0.19117647058823528
0.00031669735203066195 0.00031667657627381534
0.32783152703947355 0.3269230769230769
0.3105875487112949 0.310583883329785
0.04452237004251539 0.04452214150919008
0.14473865320319626 0.14473684210526316
0.00435325867360765 0.004353233830845771
0.5445053993810729 0.5416666666666666
0.0015173971505615732 0.001517371285056509
0.020840637105325802 0.020833333333333332
0.06250811530221385 0.0625
0.18982150993914126 0.18981481481481483
0.16144472222163359 0.1609792284866469
0.062472359663150616 0.06246553688931368
0.03262879069055642 0.03262838826562609
0.14818261920037157 0.14817880794701987
0.2699530983228547 0.2699511400651466
0.006742119077935849 0.006737918215613383
0.5414592431631918 0.5414438502673797
0.08592459396366624 0.08579335793357934
0.00019001162722080464 0.0001899030033890382
0.02301900886916793 0.022972972972972974
0.0070475396248